# Get stationary sequences from kitti scene

In [46]:
import pandas as pd
import numpy as np
import os

In [47]:
def get_data_from_dir(directory):
    data = []
    # Loop through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            # Extract the numeric part of the filename and convert it to integer
            file_index = int(filename.split('.')[0])  # Removes the extension and converts to int
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                values = file.read().strip().split()
                values = [float(value) for value in values]  # Convert each value to float
                data.append([file_index] + values)  # Add file index and values to data list
    return data

## Load data

In [48]:
# Load data
directory = '../data/2011_09_26_drive_0018_sync/oxts/data'
data = get_data_from_dir(directory)

In [49]:
columns = [
    'file_index', 'lat', 'lon', 'alt', 'roll', 'pitch', 'yaw', 
    'vn', 've', 'vf', 'vl', 'vu', 'ax', 'ay', 'az', 'af', 
    'al', 'au', 'wx', 'wy', 'wz', 'wf', 'wl', 'wu', 
    'pos_accuracy', 'vel_accuracy', 'navstat', 'numsats', 
    'posmode', 'velmode', 'orimode'
]
df = pd.DataFrame(data, columns=columns)

## Get velocity data

In [50]:
# Identify stationary frames
velocity_columns = ['vn', 've', 'vf', 'vl', 'vu']
# Set 'file_index' as the index of the DataFrame
df.set_index('file_index', inplace=True)
df.sort_index(inplace=True)  # Ensure the DataFrame is sorted by index
stationary_threshold = 0.05

## Filter for stationary frames

In [51]:
# Boolean series to identify stationary frames
is_stationary = df[velocity_columns].abs().max(axis=1) < stationary_threshold

## Get stationary sequences

In [52]:
# Find sequences of consecutive stationary frames
sequences = []
current_sequence = []
for i in df[is_stationary].index:
    if current_sequence and i == current_sequence[-1] + 1:
        current_sequence.append(i)
    else:
        if current_sequence:
            sequences.append((current_sequence[0], current_sequence[-1]))
        current_sequence = [i]
        
# Append the last sequence if it ended at the last index
if current_sequence:
    sequences.append((current_sequence[0], current_sequence[-1]))

In [53]:
# Print the sequences
for start, end in sequences:
    print(f"Stationary sequence from frame {start} to frame {end}")

Stationary sequence from frame 0 to frame 178


In [55]:
# # Optionally, if you want to see more details about each sequence:
# for start, end in sequences:
#     print(df.loc[start:end, velocity_columns + ['file_index']])